# Intro
Inference notebook for [Hotel-ID starter - classification - traning](https://www.kaggle.com/code/michaln/hotel-id-starter-classification-traning)



# Setup

In [1]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

# Imports

In [2]:
import numpy as np
import pandas as pd
import random
import os
import math

In [3]:
from PIL import Image as pil_image
from tqdm import tqdm

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Global

In [5]:
SEED = 42
IMG_SIZE = 256

PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images/"

In [7]:
print(os.listdir(PROJECT_FOLDER))

FileNotFoundError: [Errno 2] No such file or directory: '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Dataset and transformations

In [ ]:
import albumentations as A
import albumentations.pytorch as APT
import cv2

IMG_SIZE = 256

# used for training dataset - augmentations and occlusions
train_transform = A.Compose([
    A.HorizontalFlip(p=0.75),
    A.VerticalFlip(p=0.25),
    A.ShiftScaleRotate(p=0.5, border_mode=cv2.BORDER_CONSTANT),
    A.OpticalDistortion(p=0.25),
    A.Perspective(p=0.25),
    A.CoarseDropout(p=0.5, min_holes=1, max_holes=6,
                    min_height=IMG_SIZE//16, max_height=IMG_SIZE//4,
                    min_width=IMG_SIZE//16,  max_width=IMG_SIZE//4), # normal coarse dropout

    A.CoarseDropout(p=0.75, max_holes=1,
                    min_height=IMG_SIZE//4, max_height=IMG_SIZE//2,
                    min_width=IMG_SIZE//4,  max_width=IMG_SIZE//2,
                    fill_value=(255,0,0)),# simulating occlusions in test data

    A.RandomBrightnessContrast(p=0.75),
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

# used for validation dataset - only occlusions
val_transform = A.Compose([
    A.CoarseDropout(p=0.75, max_holes=1,
                    min_height=IMG_SIZE//4, max_height=IMG_SIZE//2,
                    min_width=IMG_SIZE//4,  max_width=IMG_SIZE//2,
                    fill_value=(255,0,0)),# simulating occlusions
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_31/3729967500.py:14: UserWarning: Argument(s) 'min_holes, max_holes, min_height, max_height, min_width, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(p=0.5, min_holes=1, max_holes=6,
/tmp/ipykernel_31/3729967500.py:18: UserWarning: Argument(s) 'max_holes, min_height, max_height, min_width, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(p=0.75, max_holes=1,
/tmp/ipykernel_31/3729967500.py:30: UserWarning: Argument(s) 'max_holes, min_height, max_height, min_width, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(p=0.75, max_holes=1,


In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)

    return img


def open_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = pad_image(img)
    return cv2.resize(img, (IMG_SIZE, IMG_SIZE))

In [ ]:
class HotelImageDataset:
    def __init__(self, data, transform=None, data_folder="train_images/"):
        self.data = data
        self.data_folder = data_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_folder + record["image_id"]

        image = np.array(open_and_preprocess_image(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return {
            "image" : image,
        }

# Model

In [ ]:
class HotelIdModel(nn.Module):
    def __init__(self, n_classes=100, backbone_name="resnet34"):
        super(HotelIdModel, self).__init__()

        self.backbone = timm.create_model(backbone_name, num_classes=n_classes, pretrained=False)

    def forward(self, x):
        return self.backbone(x)

# Model helper functions

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

def predict_tta(loader, model, n_matches=5, tta_transforms=3):
    model.eval()
    preds = []

    with torch.no_grad():
        for sample in tqdm(loader):
            input_images = sample['image'].to(args.device)

            tta_outputs = []
            for _ in range(tta_transforms):
                aug_input = torch.flip(input_images, dims=[-1])  # Apply horizontal flip
                outputs = model(aug_input)
                tta_outputs.append(torch.sigmoid(outputs).cpu().numpy())

            avg_outputs = np.mean(tta_outputs, axis=0)
            preds.extend(avg_outputs)

    preds = np.argsort(-np.array(preds), axis=1)[:, :n_matches]
    return preds

In [ ]:
import albumentations as A
import albumentations.pytorch as APT
import cv2

# used for training dataset - augmentations and occlusions
train_transform = A.Compose([
    A.RandomCrop(width=64, height=64),
    A.HorizontalFlip(p=0.75),
    #A.VerticalFlip(p=0.0),
    A.ShiftScaleRotate(p=0.5, shift_limit=0.0625, scale_limit=0.1, rotate_limit=10, interpolation=cv2.INTER_NEAREST, border_mode=cv2.BORDER_CONSTANT),
    A.OpticalDistortion(p=0.25, distort_limit=0.05, shift_limit=0.01),
    A.Perspective(p=0.25, scale=(0.05, 0.1)),
    A.ColorJitter(p=0.75, brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),
    A.CoarseDropout(p=0.5, min_holes=1, max_holes=5,
                    min_height=IMG_SIZE//16, max_height=IMG_SIZE//8,
                    min_width=IMG_SIZE//16,  max_width=IMG_SIZE//8), # normal coarse dropout

    A.CoarseDropout(p=0.75, max_holes=1,
                    min_height=IMG_SIZE//4, max_height=IMG_SIZE//2,
                    min_width=IMG_SIZE//4,  max_width=IMG_SIZE//2,
                    fill_value=(255,0,0)),# simulating occlusions in test data
    #A.RandomBrightnessContrast(p=0.75),
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

# used for validation dataset - only occlusions
val_transform = A.Compose([
    A.CoarseDropout(p=0.75, max_holes=1,
                    min_height=IMG_SIZE//4, max_height=IMG_SIZE//2,
                    min_width=IMG_SIZE//4,  max_width=IMG_SIZE//2,
                    fill_value=(255,0,0)),# simulating occlusions
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

# no augmentations
base_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

/tmp/ipykernel_31/3377238334.py:11: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(p=0.25, distort_limit=0.05, shift_limit=0.01),
/tmp/ipykernel_31/3377238334.py:14: UserWarning: Argument(s) 'min_holes, max_holes, min_height, max_height, min_width, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(p=0.5, min_holes=1, max_holes=5,
/tmp/ipykernel_31/3377238334.py:18: UserWarning: Argument(s) 'max_holes, min_height, max_height, min_width, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(p=0.75, max_holes=1,
/tmp/ipykernel_31/3377238334.py:29: UserWarning: Argument(s) 'max_holes, min_height, max_height, min_width, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(p=0.75, max_holes=1,


# Prepare data

In [ ]:
test_df = pd.DataFrame(data={"image_id": os.listdir(TEST_DATA_FOLDER), "hotel_id": ""}).sort_values(by="image_id")

In [ ]:
# code hotel_id mapping created in training notebook by encoding hotel_ids
hotel_id_code_df = pd.read_csv('../input/resnet-training/hotel_id_code_mapping.csv')
hotel_id_code_map = hotel_id_code_df.set_index('hotel_id_code').to_dict()["hotel_id"]

# Prepare model

In [ ]:
def get_model(model_type, backbone_name, checkpoint_path, args):
    model = HotelIdModel(args.n_classes, backbone_name)

    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint["model"])
    model = model.to(args.device)

    return model

In [ ]:
class args:
    batch_size = 64
    num_workers = 2
    n_classes = hotel_id_code_df["hotel_id"].nunique()
    device = ('cuda' if torch.cuda.is_available() else 'cpu')


seed_everything(seed=SEED)

test_dataset = HotelImageDataset(test_df, base_transform, data_folder=TEST_DATA_FOLDER)
test_loader = DataLoader(test_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

In [ ]:
import torch

def get_model(model_type, backbone_name, checkpoint_path, args):
    model = HotelIdModel(args.n_classes, backbone_name)

    # Load the checkpoint with map_location
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

    model.load_state_dict(checkpoint["model"])
    model = model.to(args.device)  # Ensure it's moved to the correct device (CPU/GPU)

    return model

In [ ]:
model = get_model("classification", "resnet34",
                  "../input/resnet-training/checkpoint-classification-model-resnet34-256x256.pt",
                  args)

print(model)


/tmp/ipykernel_31/1137292120.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


HotelIdModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding

# Submission

In [ ]:
%%time

preds = predict_tta(test_loader, model, n_matches=5, tta_transforms=3)
# replace classes with hotel_id using mapping created in trainig notebook
preds = [[hotel_id_code_map[b] for b in a] for a in preds]
# transform array of hotel_ids into string
test_df["hotel_id"] = [str(list(l)).strip("[]").replace(",", "") for l in preds]

test_df.to_csv("submission.csv", index=False)
test_df.head()

100%|██████████| 1/1 [00:00<00:00,  1.15it/s]

CPU times: user 1.3 s, sys: 181 ms, total: 1.48 s
Wall time: 884 ms


,image_id,hotel_id
0,abc.jpg,24700 108817 73834 18800 203929


## Gradio

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
!ls "/content/drive/My Drive/Zinnia Gradio"

'Copy of hotel_resnet34.pkl'   hotel_id_code_mapping.csv   submission.csv
 gradio.ipynb		       hotel_resnet34.pkl	  'Test Image.jpg'


In [14]:
# 1) Point at your Drive folder
DRIVE_FOLDER      = "/content/drive/My Drive/Zinnia Gradio"
WEIGHTS_PATH      = f"{DRIVE_FOLDER}/Copy of hotel_resnet34.pkl"
MAPPING_CSV_PATH  = f"{DRIVE_FOLDER}/hotel_id_code_mapping.csv"



In [16]:
# 2) Reload everything from Drive
import timm, pandas as pd, numpy as np, cv2
from albumentations import Compose, ToFloat
from albumentations.pytorch import ToTensorV2
from PIL import Image
import torch


In [17]:
# 2a) load your hotel_id mapping
mapping_df        = pd.read_csv(MAPPING_CSV_PATH)
hotel_id_code_map = mapping_df.set_index("hotel_id_code")["hotel_id"].to_dict()


In [19]:
# 2b) redefine your model class
class HotelIdModel(torch.nn.Module):
    def __init__(self, n_classes, backbone_name="resnet34"):
        super().__init__()
        self.backbone = timm.create_model(backbone_name,
                                          num_classes=n_classes,
                                          pretrained=False)
    def forward(self, x):
        return self.backbone(x)

In [20]:
# 2c) instantiate + load weights
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = len(hotel_id_code_map)
model    = HotelIdModel(n_classes, "resnet34")
state    = torch.load(WEIGHTS_PATH, map_location=device)
model.load_state_dict(state)
model.to(device).eval()
print("✅ Model reloaded and ready")


✅ Model reloaded and ready


In [22]:
# 3) your preprocessing & TTA predict helper (unchanged)
base_transform = Compose([ ToFloat(), ToTensorV2() ])

def pad_and_resize(img, size=256):
    h, w = img.shape[:2]
    diff = abs(h - w)//2
    if h > w:
        img = cv2.copyMakeBorder(img, 0,0, diff,diff, cv2.BORDER_CONSTANT)
    else:
        img = cv2.copyMakeBorder(img, diff,diff,0,0, cv2.BORDER_CONSTANT)
    return cv2.resize(img, (size,size))

def predict_top5(pil_img):
    img    = np.array(pil_img)                # PIL→RGB numpy
    img    = pad_and_resize(img, size=256)
    tensor = base_transform(image=img)["image"].unsqueeze(0).to(device)
    with torch.no_grad():
        out1 = torch.sigmoid(model(tensor))
        out2 = torch.sigmoid(model(torch.flip(tensor, dims=[-1])))
        avg  = (out1 + out2)/2
    idxs = torch.topk(avg, k=5, dim=1).indices.cpu().numpy().ravel()
    return [hotel_id_code_map[int(i)] for i in idxs]

In [23]:
!pip install gradio

In [26]:
import gradio as gr

iface = gr.Interface(
    fn=predict_top5,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.JSON(label="Top 5 Hotel IDs"),
    title="Hotel-ID Classifier",
    description="Upload a hotel room photo and get the top-5 predicted hotel IDs."
)
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0a361a02a1691b8d2b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
